#### Include the following in actions.json file for the prompt template

In [ ]:
{
        "name": "GetMargiesTravelInfo",
        "description": "Gets information about Margie's Travel from Azure AI Search's index and vectorstore",
        "parameters": {
            "type": "object",
            "properties": {
                "user_query": {
                    "type": "string",
                    "description": "the query of the user about Margie's Travel Agency"
                }
            },
            "required": [
                "user_query"
            ]
        }
}

#### Include the following import statements

In [ ]:
import requests
import openai
from openai import AzureOpenAI
import json

#### Embedding Generator Function - in bot.py file

In [ ]:
def generate_embeddings(client, text):
    embedding_model = "YOUR_EMBEDDING_MODEL_NAME"
    
    response = client.embeddings.create(
        input=text,
        model = embedding_model
    )
    
    embeddings=response.model_dump()
    return embeddings['data'][0]['embedding']

#### The AI Action code - in bot.py file


In [ ]:
@bot_app.ai.action("GetMargiesTravelInfo")
async def get_margies_travel_info(context: TurnContext, state: TurnState):
    
    # Fetching the user query from the context
    user_query = str(context.data["user_query"]) if context.data["user_query"] else "null"
    
    # Creating an instance of the AzureOpenAI client
    azure_openai_client=AzureOpenAI(
        api_key = config.AZURE_OPENAI_API_KEY,
        api_version = "2024-02-15-preview",
        azure_endpoint = config.AZURE_OPENAI_ENDPOINT
    )
    
    # Generating vector embeddings for the user query
    vectorised_user_query = generate_embeddings(azure_openai_client, user_query)
    
    url = f"YOUR_AZURE_AI_SEARCH_ENDPOINT/indexes/YOUR_INDEX_NAME/docs/search?api-version=2023-11-01"
    
    headers = {
            "Content-Type": "application/json",
            "api-key": "YOUR_AI_SEARCH_API_KEY"
        }
        
    body =   {
            "count": True,
            "select": "chunk",
            "vectorQueries": [
                {
                    "vector": vectorised_user_query,
                    "k": 3,
                    "fields": "text_vector",
                    "kind": "vector"
                }
            ]
        }
        
    response = requests.post(url, headers=headers, data=json.dumps(body))
    documents = response.json()['value']
    
    grounding_knowledge = []
    
    for doc in documents:
        grounding_knowledge.append(dict(
            {
                "chunk": doc['chunk'],
                "score": doc['@search.score']
                
            }
        ))
    
    await context.send_activity(f"fetching supporting knowledge for your query")
    return f"the grounding knowledge is {grounding_knowledge}" 